In [41]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('ab_data.csv', sep=';')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [3]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


## Посмотрим, есть ли пропущенные значения в данных

In [4]:
df.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

## Посмотрим, есть ли дубликаты в пользователях

In [5]:
df.user_id.nunique()
df.shape[0]

294478

## Есть дубликаты в user_id

In [6]:
users = df.groupby('user_id').size()
users[users>1]

user_id
630052    2
630126    2
630137    2
630320    2
630471    2
         ..
945627    2
945645    2
945703    2
945797    2
945971    2
Length: 3894, dtype: int64

In [7]:
df.drop_duplicates(subset='user_id', keep='first', inplace=True)

In [8]:
df.user_id.nunique() == df.shape[0]

True

## Проверим группы control и treatment

In [23]:
pd.crosstab(df['group'], df['landing_page']).reset_index()

landing_page,group,new_page,old_page
0,control,0,144226
1,treatment,144314,0


## controll group видят new_page. Нужно это исправить

In [10]:
df=df[~((df.group=='control')&(df.landing_page=='new_page'))]
df=df[~((df.group=='treatment')&(df.landing_page=='old_page'))]

In [21]:
pd.crosstab(df['group'], df['landing_page']).reset_index()

landing_page,group,new_page,old_page
0,control,0,144226
1,treatment,144314,0


## Нулевая гипотеза: Нет значительной разницы между двумя группами
## Альтернативная гипотеза: Изменения улучшат конверсию

## Разобъём на 2 группы

In [12]:
group_a = df[df['group'] == 'control']['converted']
group_b = df[df['group'] == 'treatment']['converted']

## Посчитаем среднее, дисперсию и размер выборки у каждой группы

In [13]:
mean_a = np.mean(group_a)
mean_b = np.mean(group_b)

In [14]:
var_a = np.var(group_a, ddof=1)
var_b = np.var(group_b, ddof=1)

In [15]:
n_a = len(group_a)
n_b = len(group_b)

In [16]:
df.groupby(['group','landing_page'])['converted'].agg(['count','sum','mean']).reset_index()

,group,landing_page,count,sum,mean
0,control,old_page,144226,17349,0.120290
1,treatment,new_page,144314,17134,0.118727


## Проведём z-тест

In [51]:
nobs = n_a, n_b
successes = df[df.landing_page == 'old_page']['converted'].sum(), df[df.landing_page == 'new_page']['converted'].sum()

In [52]:
z, p = proportions_ztest(count=successes, nobs=nobs)
print('Z Stat = %.4f, p-value = %.4f' % (z, p))

Z Stat = 1.2942, p-value = 0.1956


In [48]:
alpha = 0.05
if p < alpha:
    print("Различия между выборками статистически значимы")
else:
    print("Нет статистически значимых различий между выборками")

Нет статистически значимых различий между выборками


## p-value выше 0.05, значит мы не можем отклонить нулевую гипотезу